In [1]:
import numpy as np
import datetime as dt
from astropy.time import Time
from astropy.coordinates import get_sun,Angle
from astropy import units as u
from astropy.coordinates import SkyCoord
from prettytable import PrettyTable

# Orbital Elements

In [2]:
def hms2rad(hm_list):
    h,m,s= hm_list[0],hm_list[1],hm_list[2]
    ang = str(int(h))+'h'+str(int(m))+'m'+str(s)+'s'
    rad = Angle(ang).to(u.rad).value
    return rad

In [3]:
def dms2rad(dm_list):
    d,m,s = dm_list[0],dm_list[1],dm_list[2]
    ang = str(int(d))+'d'+str(int(m))+'m'+str(s)+'s'
    rad = Angle(ang).to(u.rad).value
    return rad

In [4]:
def cook_data(data):
    dates = data[0]
    ra_raw = data[1]
    dec_raw = data[2]
    ra = []
    for i in ra_raw:
        ra_i = list(map(float, i.split(' ')))
        ra.append(ra_i)
    dec = []
    for i in dec_raw:
        dec_i = list(map(float, i.split(' ')))
        dec.append(dec_i)
    ra = np.array(ra)
    dec = np.array(dec)
    ra_rad = []
    dec_rad = []
    for i in range(len(ra)):
        h,m,s= ra[i][0],ra[i][1],ra[i][2]
        ang_hms = Angle(str(int(h))+'h'+str(int(m))+'m'+str(s)+'s')
        rad1 = ang_hms.to(u.rad).value
        ra_rad.append(rad1)
        d,mi,se= dec[i][0],dec[i][1],dec[i][2]
        ang_dms = Angle(str(int(d))+'d'+str(int(mi))+'m'+str(se)+'s')
        rad2 = ang_dms.to(u.rad).value
        dec_rad.append(rad2)
    ra_rad,dec_rad = np.array(ra_rad),np.array(dec_rad)
    time = []
    for i in dates:
        date = Time.strptime(i, "%Y-%b-%d",scale='utc')
        t = date.tt
        time.append(t)
    time = np.array(time)
    return ra_rad,dec_rad,time

In [5]:
def round_array(array,N):
    x = []
    for i in range(len(array)):
        m = round(array[i],N)
        x.append(m)
        
    return np.array(x)

In [6]:
def makePrettyTable(table_col1, table_col2, table_col3):
    """ For table Output """
    table = PrettyTable()
    table.add_column('',round_array(table_col1,6))
    table.add_column('',round_array(table_col2,6))
    table.add_column('',table_col3)
    return print(table)

In [7]:
def tablefortwo(table_col1, table_col2):
    """ For table Output """
    table = PrettyTable()
    table.add_column('',round_array(table_col1,6))
    table.add_column('',round_array(table_col2,6))
    return print(table)

In [8]:
def eq2cart(ra,dec,dist): # equatorial to Cartesian
    x = dist*np.cos(ra)*np.cos(dec)
    y = dist*np.sin(ra)*np.cos(dec)
    z = dist*np.sin(dec)

    return x,y,z

In [9]:
def sunattime(time):
    ra_s,dec_s,dist_s = np.array([]),np.array([]),np.array([])

    for i in time:
        ra_s = np.insert(ra_s,len(ra_s),get_sun(i).ra.value,axis=0)
        dec_s = np.insert(dec_s,len(dec_s),get_sun(i).dec.value,axis=0)
        dist_s = np.insert(dist_s,len(dist_s),get_sun(i).distance.value,axis=0)
    ra_s,dec_s = np.deg2rad(ra_s),np.deg2rad(dec_s)
    x_s,y_s,z_s = eq2cart(ra_s,dec_s,dist_s)
    return x_s,y_s,z_s

In [10]:
def cooktime(t1,t2,t3):
    T1 = (t3-t2).value/58.13244087
    T2 = (t3-t1).value/58.13244087
    T3 = (t2-t1).value/58.13244087
    return T1,T2,T3

In [11]:
def lmn(ra,dec):
    l = np.cos(ra)*np.cos(dec)
    m = np.sin(ra)*np.cos(dec)
    n = np.sin(dec)
    return l,m,n

In [12]:
def distances(l,m,n,x_s,y_s,z_s,a1,b1,a3,b3):
    l1,l2,l3 = l[0],l[1],l[2]
    m1,m2,m3 = m[0],m[1],m[2]
    n1,n2,n3 = n[0],n[1],n[2]
    x_s1, x_s2, x_s3 = x_s[0],x_s[1],x_s[2]
    y_s1, y_s2, y_s3 = y_s[0],y_s[1],y_s[2]
    z_s1, z_s2, z_s3 = z_s[0],z_s[1],z_s[2]
    
    A = [[l1*a1,-l2,l3*a3],[m1*a1,-m2,m3*a3],[n1*a1,-n2,n3*a3]]
    B = [a1*x_s1 - x_s2 + a3*x_s3, a1*y_s1 - y_s2 + a3*y_s3, a1*z_s1 - z_s2 + a3*z_s3]
    
    try:
        Deltas = np.linalg.solve(A,B)
    except:
        Deltas = np.linalg.lstsq(A,B,rcond=None)
        
    xi = l*Deltas - x_s
    eta = m*Deltas - y_s
    zeta = n*Deltas - z_s
    r = np.sqrt(xi**2 + eta**2 + zeta**2)
#    tablefortwo(Deltas,r)
    return Deltas,r

In [13]:
def improve_a(b1,b3,T1,T3,r):
    a1 = b1 + (T1*T3/(6*(r[1]**3)))*(1+b1)
    a3 = b3 + (T1*T3/(6*(r[1]**3)))*(1+b3)
    return a1,a3

In [14]:
def lt_correction(D,t1,t2,t3):
    # Light-time correction

    c = 10065.320
    t1_d = D[0]/c
    t1 = t1 - t1_d
    t2_d = D[1]/c
    t2 = t2 - t2_d
    t3_d = D[2]/c
    t3 = t3 - t3_d
    return t1,t2,t3

In [15]:
def calcgeo(l,m,n,D,x_s,y_s,z_s):
    xi = l*D - x_s
    eta = m*D - y_s
    zeta = n*D - z_s

    return xi,eta,zeta

In [16]:
def calcf3(r,xi,eta,zeta):
    xi1, xi2, = xi[0], xi[1]
    eta1, eta2 = eta[0], eta[1]
    zeta1, zeta2 = zeta[0], zeta[1]
    r1, r2 = r[0], r[1]
    cos2f3 = (xi1*xi2 + eta1*eta2 + zeta1*zeta2)/(r1*r2)
    cosf3 = np.sqrt((1+cos2f3)/2)
    f3 = np.arccos(cosf3)
    return f3

In [17]:
def calcf2(r,xi,eta,zeta):
    xi1, xi3 = xi[0], xi[2]
    eta1, eta3 = eta[0], eta[2]
    zeta1, zeta3 = zeta[0], zeta[2]
    r1, r3 = r[0], r[2]
    cos2f2 = (xi3*xi1 + eta3*eta1 + zeta3*zeta1)/(r3*r1)
    cosf2 = np.sqrt((1+cos2f2)/2)
    f2 = np.arccos(cosf2)
    return f2

In [18]:
def calcf1(r,xi,eta,zeta):
    xi2, xi3 = xi[1], xi[2]
    eta2, eta3 = eta[1], eta[2]
    zeta2, zeta3 = zeta[1], zeta[2]
    r2, r3 = r[1], r[2]
    cos2f1 = (xi2*xi3 + eta2*eta3 + zeta2*zeta3)/(r2*r3)
    cosf1 = np.sqrt((1+cos2f1)/2)
    f1 = np.arccos(cosf1)
    return f1

In [19]:
def calcM2N(T3,f3,r1,r2):
    M3 = T3/(2*((np.sqrt(r1*r2)*np.cos(f3))**(3/2)))
    N3 = (r1+r2)/(2*(np.sqrt(r1*r2)*np.cos(f3)))
    M32 = M3**2
    return M32,N3

In [20]:
def functionsolve(a,b,xg,yg):
    count=0
    x,y = 10,10
    h,k=1,1
    while abs((x-xg)/xg)>1e-15 and abs((y-yg)/yg)>1e-15:
        f = (xg**2 - a/(b-np.cos(yg)))
        g = (xg**3 - xg**2 - ((a*(yg-np.sin(yg)*np.cos(yg)))/(np.sin(yg)**3)))
        fx = 2*xg
        fy = (a*np.sin(yg))/((b-np.cos(yg))**2)
        gx = xg*(3*xg-2)
        gy = ((a*(3*(yg-np.sin(yg)*np.cos(yg))*np.cos(yg) - 2*np.sin(yg)**3))/(np.sin(yg)**4))
        h,k = np.linalg.solve([[fx,fy],[gx,gy]],[f,g])
        x,y = xg,yg
        xg = xg-h
        yg = yg-k
        count +=1
#         print(x,y)
#     print(count,(x-xg)/xg,(y-yg)/yg)
    return x,y

In [21]:
def functionsolve_guess(M,N,f):
    g = f
    R = 1
    R3,g3 = functionsolve(M,N,R,g)
    return R3,g3

In [22]:
def calc_e_nu(r1,r2,r3,f1,f2,f3,l):
    ecosnu1 = l/r1 - 1
    esinnu1 = (np.cos(2*f2)*(l/r1 - 1) - (l/r3 - 1))/(np.sin(2*f2))
    nu1 = np.arctan2(esinnu1,ecosnu1)
    # e = ecosnu1/np.cos(nu1)
#    nu1,nu2,nu3 = nu1+2*np.pi,nu2+2*np.pi,nu3+2*np.pi
    e = esinnu1/np.sin(nu1)
    nu1 = check_angle_rad(nu1)
    nu2 = nu1 + 2*f3
    nu3 = nu1 + 2*f2
    return e,nu1,nu2,nu3

In [23]:
def calc_a(l,e):
    a = l/(1-e*e)
    return a

In [24]:
def calc_E(e,nu):
    cos = (e+np.cos(nu))/(1+(e*np.cos(nu)))
    sin = ((np.sin(nu)*np.sqrt(1 - e**2)))/(e+np.cos(nu))
    E = np.arccos(cos)
    #E = np.arctan2(sin,cos)
    if (cos<0)and(sin<0):
        E = E+(np.pi/2)
    elif (cos>0)and(sin<0):
        E = E+np.pi
    else:
        E = E
    return E

In [25]:
def PQ1(xi,eta,zeta,r,f1,f2,f3,nu1,nu2,nu3):
    xi1, xi3 = xi[0], xi[2]
    eta1, eta3 = eta[0], eta[2]
    zeta1, zeta3 = zeta[0], zeta[2]

    r1,r3 = r[0],r[2]

    Px = (xi1*r3*np.sin(nu3) - xi3*r1*np.sin(nu1))/(r1*r3*np.sin(2*f2))
    Qx = (xi3*r1*np.cos(nu1) - xi1*r3*np.cos(nu3))/(r1*r3*np.sin(2*f2))

    Py = (eta1*r3*np.sin(nu3) - eta3*r1*np.sin(nu1))/(r1*r3*np.sin(2*f2))
    Qy = (eta3*r1*np.cos(nu1) - eta1*r3*np.cos(nu3))/(r1*r3*np.sin(2*f2))

    Pz = (zeta1*r3*np.sin(nu3) - zeta3*r1*np.sin(nu1))/(r1*r3*np.sin(2*f2))
    Qz = (zeta3*r1*np.cos(nu1) - zeta1*r3*np.cos(nu3))/(r1*r3*np.sin(2*f2))

    return Px,Py,Pz,Qx,Qy,Qz

In [26]:
def calc_omega(Py,Pz,Qy,Qz):
    eps = 23.439291
    epsrad = np.deg2rad(eps)
    sin = Pz*np.cos(epsrad) - Py*np.sin(epsrad)
    cos = Qz*np.cos(epsrad) - Qy*np.sin(epsrad)
    omega = np.arctan2(sin,cos)
#     print(np.rad2deg(omega))
    return omega

In [27]:
def calc_OMega(Px,Py,Qx,Qy,omega):
    eps = 23.439291
    epsrad = np.deg2rad(eps)
    sin = (Py*np.cos(omega) - Qy*np.sin(omega))*(1/np.cos(epsrad))
    cos = Px*np.cos(omega) - Qx*np.sin(omega)
    OMega = np.arctan2(sin,cos)
#     print(np.rad2deg(OMega))
    return OMega

In [28]:
def calc_i(Px,Qx,omega,OMega):
    cos = -(Px*np.sin(omega)+Qx*np.cos(omega))*(1/np.sin(OMega))
    i = np.arccos(cos)
#     print(np.rad2deg(i))
    return i

In [29]:
def improve_iterate(b1,b3,T1,T3,r,l,m,n,x_s,y_s,z_s):
    prec = 1
    count=0
    while prec>1e-15:
        r2_old = r[1]
        a1,a3 = improve_a(b1,b3,T1,T3,r)
        D,r = distances(l,m,n,x_s,y_s,z_s,a1,b1,a3,b3)
        r2_new = r[1]
        prec = abs((r2_new - r2_old)/r2_old)
        count +=1
#        print(D,r)
#    print(count)
    return a1,a3,D,r

In [30]:
def check_angle_rad(rad):
    if rad<0:
        if abs(rad)<2*np.pi:
            rad = rad + 2*np.pi
        if abs(rad)>2*np.pi:
            rad = abs(rad + 2*np.pi)
    if rad>0:
        if rad>2*np.pi:
            rad = rad - 2*np.pi    
    return rad

In [31]:
def check_angle_deg(deg):
    if deg<0:
        if abs(deg)<360:
            deg = deg + 360
        if abs(deg)>360:
            deg = abs(deg + 360)
    if deg>0:
        if deg>360:
            deg = deg - 360
    return deg

In [32]:
def days2syr(days):
    days = days/365.25636
    return days

In [33]:
def error_check(a,e,i,omega,OMega,P):
    elts0 = get_elements(main_file)
    a0 = elts0[0]
    e0 = elts0[1]
    i0 = elts0[2]
    OMega0 = elts0[3]
    omega0 = elts0[4]
    P0 = elts0[5]
    i0 = Angle(i0*u.deg).rad
    omega0 = Angle(omega0*u.deg).rad
    OMega0 = Angle(OMega0*u.deg).rad
    acc_a = round((1 - (abs((a-a0)/a0)))*100,3)
    acc_e = round((1 -(abs((e-e0)/e0)))*100,3)
    acc_i = round((1 -(abs((i-i0)/i0)))*100,3)
    acc_om = round((1 -(abs((omega-omega0)/omega0)))*100,3)
    acc_Om = round((1 -(abs((OMega-OMega0)/OMega0)))*100,3)
    acc_P = round((1 -(abs((P-P0)/P0)))*100,3)
    return acc_a,acc_e,acc_i,acc_om,acc_Om,acc_P

# Ephemeris

In [34]:
import astropy.units as u
from astropy.coordinates import Angle,get_sun
from astropy.time import Time
import numpy as np

In [35]:
def time_array(t0,N,step):
    t_arr = []
    for i in range(1,N+1):
        t_arr.append(t0+i*step)
    return t_arr

In [36]:
def Period(a):
    P = a**(1.5)
    return P

In [37]:
def MA(M0,t_arr,t0,P):
    M0 = M0.rad
    P = P*365.25636
    M_arr = []
    for i in range(len(t_arr)):
        M = M0 + ((2*np.pi)/P)*((t_arr[i]-t0).value)
        M_arr.append(M)
    return M_arr

In [38]:
def Kep_Solve(e,M):
    E_arr = []
    for i in range(len(M)):
        x = (e*np.sin(M[i]))/(1-e*np.cos(M[i]))
        Eg = M[i] + x*(1-0.5*x**2)
        prec = 1
        count = 0
        while prec>1e-10:
            E = (M[i] - e*(Eg*np.cos(Eg) - np.sin(Eg)))/(1-e*np.cos(Eg))
            prec = abs((Eg-E)/Eg)
            Eg = E
        E_arr.append(E)
    return E_arr

In [39]:
def PQ_eph(com,om,i):
    ob = np.deg2rad(23.439291)
    com = np.deg2rad(com)
    om = np.deg2rad(om)
    i = np.deg2rad(i)
    Px = np.cos(com)*np.cos(om) - np.sin(com)*np.sin(om)*np.cos(i)
    Qx = -(np.cos(com)*np.sin(om) + np.sin(com)*np.cos(om)*np.cos(i))
    Py = (np.sin(com)*np.cos(om) + np.cos(com)*np.sin(om)*np.cos(i))*np.cos(ob) - np.sin(om)*np.sin(i)*np.sin(ob)
    Qy = (-np.sin(com)*np.sin(om) + np.cos(com)*np.cos(om)*np.cos(i))*np.cos(ob) - np.cos(om)*np.sin(i)*np.sin(ob)
    Pz = (np.sin(com)*np.cos(om) + np.cos(com)*np.sin(om)*np.cos(i))*np.sin(ob) + np.sin(om)*np.sin(i)*np.cos(ob)
    Qz = (-np.sin(com)*np.sin(om) + np.cos(com)*np.cos(om)*np.cos(i))*np.sin(ob) + np.cos(om)*np.sin(i)*np.cos(ob)
    
    return Px,Qx,Py,Qy,Pz,Qz

In [40]:
def helio_eq(a,Px,Py,Pz,Qx,Qy,Qz,E,e):
    b = a*np.sqrt(1-e**2)
    xi,eta,zeta = [],[],[]
    for i in range(len(E)):
        Xi = a*Px*(np.cos(E[i]) - e) + b*Qx*(np.sin(E[i]))
        Eta = a*Py*(np.cos(E[i]) - e) + b*Qy*(np.sin(E[i]))
        Zeta = a*Pz*(np.cos(E[i]) - e) + b*Qz*(np.sin(E[i]))
        xi.append(Xi)
        eta.append(Eta)
        zeta.append(Zeta)
    return xi,eta,zeta

In [41]:
def polar2cart(ra,dec,dist):
    dec,ra = np.deg2rad(dec),np.deg2rad(ra)
    x = dist*np.cos(dec)*np.cos(ra)
    y = dist*np.cos(dec)*np.sin(ra)
    z = dist*np.sin(dec)
    return x,y,z

In [42]:
def geo_cart(xi,eta,zeta,T_arr):
    X,Y,Z = [],[],[]
    for i in range(len(xi)):
        ra_s = get_sun(T_arr[i]).ra.value
        dec_s = get_sun(T_arr[i]).dec.value
        dist_s = get_sun(T_arr[i]).distance.value
        x_s,y_s,z_s = polar2cart(ra_s,dec_s,dist_s)
        x = x_s + xi[i]
        y = y_s + eta[i]
        z = z_s + zeta[i]
        X.append(x)
        Y.append(y)
        Z.append(z)
    return X,Y,Z

In [43]:
def cart2polar(x_arr,y_arr,z_arr):
    ra,dec,dist = [],[],[]
    for i in range(len(x_arr)):
        cosra = x_arr[i]/(np.sqrt(x_arr[i]**2 + y_arr[i]**2))
        sinra = y_arr[i]/(np.sqrt(x_arr[i]**2 + y_arr[i]**2))
        
        Ra = np.arctan2(sinra,cosra)
        
        Dec = np.arcsin(z_arr[i]/np.sqrt(x_arr[i]**2 + y_arr[i]**2 + z_arr[i]**2))
        
        Dist = np.sqrt(x_arr[i]**2 + y_arr[i]**2 + z_arr[i]**2)
        
        ra.append(Ra)
        dec.append(Dec)
        dist.append(Dist)
        
    return ra,dec,dist

In [44]:
def ra2hms(ra):
    hms_ra = []
    for i in range(len(ra)):
        h = Angle(ra[i]*u.rad)
        if h<0:
            h = Angle(2*np.pi*u.rad) + h
        h = h.hms
        h = str(int(h.h))+' '+str(int(h.m))+' '+str(round(h.s,4))
        hms_ra.append(h)
    return hms_ra

def dec2dms(dec):
    dms_dec = []
    for i in range(len(dec)):
        d = Angle(dec[i]*u.rad).dms
        d = str(int(d.d)) +' '+str(int(abs(d.m)))+' '+str(round(abs(d.s),4))
        dms_dec.append(d)
    return dms_dec

def dist2str(dist):
    dist_str = []
    for i in range(len(dist)):
        dist_str.append(str(round(dist[i],4)))
    return dist_str

In [45]:
def ephemeris(a,e,i,OM,w,M0,t0,N,step):
    P = Period(a.value)
    T_arr = time_array(t0,N,step)
    tf = T_arr[-1] # final time
    M = MA(M0,T_arr,t0,P)
    E = Kep_Solve(e,M)
    Px,Qx,Py,Qy,Pz,Qz = PQ_eph(OM.value,w.value,inc.value)
    xi,eta,zeta = helio_eq(a.value,Px,Py,Pz,Qx,Qy,Qz,E,e)
    x,y,z = geo_cart(xi,eta,zeta,T_arr)
    ra,dec,dist = cart2polar(x,y,z)
    ra = ra2hms(ra)
    dec = dec2dms(dec)
    dist = dist2str(dist)
    return ra,dec,dist,T_arr

In [46]:
def time2str(arr):
    T_str = []
    for i in range(len(arr)):
        y = arr[i].strftime("%Y-%b-%d")
        T_str.append(y)
    T_str = np.array(T_str)
    return T_str

In [47]:
def data_import(filename,sep):
    a_file = open(filename)
    data_lines_to_read = []
    for i in range(72,98):
        data_lines_to_read.append(i)

    x = []
    for position, line in enumerate(a_file):
        if position in data_lines_to_read:
            x.append(line)

    date,ra,dec,delta = [],[],[],[]
    for i in x:
        Date = i[1:12]
        Ra = i[23:34]
        Dec = i[35:46]
        Delta = i[64:79]
        date.append(Date)
        ra.append(Ra)
        dec.append(Dec)
        delta.append(Delta)
    dec_raw = np.array(dec[::-sep][:3][::-1])
    ra_raw = np.array(ra[::-sep][:3][::-1])
    date_raw = np.array(date[::-sep][:3][::-1])
    data = np.array([date_raw,ra_raw,dec_raw])
    return data

In [48]:
def get_elements(main_file):
    a_file = open(main_file)
    lines_to_read = [7,8, 9,10,12]
    x = []
    for position, line in enumerate(a_file):
        if position in lines_to_read:
            x.append(line)

    oList = []
    for i in x:
        p = i.split('   ')
        p = list(filter(('').__ne__, p))
        oList.append(p[0])
        oList.append(p[1])
        oList.append(p[2])

    OList = list(filter(('\n').__ne__, oList))
    delet = [1,2,7,8,10,11,12,13,14]
    OList = OList[0:-5:1]
    del OList[2]
    del OList[-2]
    del OList[-2]
    del OList[1]

    elts_arr = []
    for i in OList:
        k = i.split('= ')
        elts_arr.append(k[1])

    e0,OM0,w0,inc0,a0,P0 = elts_arr[0],elts_arr[1],elts_arr[2],elts_arr[3],elts_arr[4],elts_arr[5]
    a0 = float(a0)
    e0 = float(e0)
    inc0 = float(inc0)
    OM0 = float(OM0)
    w0 = float(w0)
    P0 = float(P0)
    elts_arr = np.array([a0,e0,inc0,OM0,w0,P0])
    return elts_arr

In [49]:
# MAIN CODE
main_file = '2019-QB3.txt'
num = main_file[-7:-4] # the name of the object
#data = np.genfromtxt(in_file,delimiter=';',dtype=str)
data = data_import(main_file,3)
ra_rad,dec_rad,dates = cook_data(data)
#print(ra_rad,dec_rad,dates)
t1,t2,t3 = dates[0],dates[1],dates[2]
t10 = dates[0]
# STEP 3 : Solar coordinates
x_s, y_s, z_s = sunattime(dates)

T1,T2,T3 = cooktime(t1,t2,t3)
b1 = T1/T2
b3 = T3/T2

# STEP 4 : Geocentric direction cosines
l,m,n = lmn(ra_rad,dec_rad)

a1 = b1
a3 = b3
# STEP 6 : Geocentric and Heliocentric distances
# Delta / D = Geocentric distance
# r = heliocentric distance

D,r = distances(l,m,n,x_s,y_s,z_s,a1,b1,a3,b3)
# tablefortwo(D,r)

# STEP 7 : Improve a1 and a3
a1,a3,D,r = improve_iterate(b1,b3,T1,T3,r,l,m,n,x_s,y_s,z_s)
xi,eta,zeta = calcgeo(l,m,n,D,x_s,y_s,z_s)

# STEP 9 : Light-time corrections
t1,t2,t3 = lt_correction(D,t1,t2,t3)
correct_time = np.array([t1,t2,t3])
T1,T2,T3 = cooktime(t1,t2,t3)


# R:STEP 3:
x_s, y_s, z_s = sunattime(correct_time)

# R:STEP 6,7:
D,r = distances(l,m,n,x_s,y_s,z_s,a1,b1,a3,b3)
# tablefortwo(D,r)

xi,eta,zeta = calcgeo(l,m,n,D,x_s,y_s,z_s)


# STEP 10 : Calculate f1,f2,f3, M^2 (3 values) and N (3 values)
f3 = calcf3(r,xi,eta,zeta)
f2 = calcf2(r,xi,eta,zeta)
f1 = calcf1(r,xi,eta,zeta)

r1, r2, r3 = r[0], r[1], r[2]
M32,N3 = calcM2N(T3,f3,r1,r2)
M22,N2 = calcM2N(T2,f2,r3,r1)
M12,N1 = calcM2N(T1,f1,r2,r3)

R3,g3 = functionsolve_guess(M32,N3,f3)
R2,g2 = functionsolve_guess(M22,N2,f2)
R1,g1 = functionsolve_guess(M12,N1,f1)

# STEP 11 : New triangle ratios
a1 = (R2/R1)*b1
a3 = (R2/R3)*b3


D,r = distances(l,m,n,x_s,y_s,z_s,a1,b1,a3,b3)
t1,t2,t3 = lt_correction(D,t1,t2,t3)
correct_time = np.array([t1,t2,t3])
T1,T2,T3 = cooktime(t1,t2,t3)
x_s, y_s, z_s = sunattime(correct_time)
D,r = distances(l,m,n,x_s,y_s,z_s,a1,b1,a3,b3)
xi,eta,zeta = calcgeo(l,m,n,D,x_s,y_s,z_s)
f3 = calcf3(r,xi,eta,zeta)
f2 = calcf2(r,xi,eta,zeta)
f1 = calcf1(r,xi,eta,zeta)
r1, r2, r3 = r[0], r[1], r[2]
M32,N3 = calcM2N(T3,f3,r1,r2)
M22,N2 = calcM2N(T2,f2,r3,r1)
M12,N1 = calcM2N(T1,f1,r2,r3)
R3,g3 = functionsolve(M32,N3,R3,g3)
R2,g2 = functionsolve(M22,N2,R2,g2)
R1,g1 = functionsolve(M12,N1,R1,g1)


l = (R3*r1*r2*np.sin(2*f3)/T3)**2

e,nu1,nu2,nu3 = calc_e_nu(r1,r2,r3,f1,f2,f3,l)
nu1,nu2,nu3 = check_angle_rad(nu1),check_angle_rad(nu2),check_angle_rad(nu3)
a = calc_a(l,e)

P = (a**(3/2))*365.25636

E1,E2,E3 = calc_E(e,nu1),calc_E(e,nu2),calc_E(e,nu3)

Px,Py,Pz,Qx,Qy,Qz = PQ1(xi,eta,zeta,r,f1,f2,f3,nu1,nu2,nu3)

omega = calc_omega(Py,Pz,Qy,Qz)
omega = check_angle_rad(omega)

OMega = calc_OMega(Px,Py,Qx,Qy,omega)
OMega = check_angle_rad(OMega)

i_rad = calc_i(Px,Qx,omega,OMega)

nu1,nu2,nu3 = check_angle_rad(nu1),check_angle_rad(nu2),check_angle_rad(nu3)
T = (P/(2*np.pi))*(E1-e*np.sin(E1))
T_p = (t1 - T)

acc_a, acc_e, acc_i, acc_w,acc_OM,acc_P = error_check(a,e,i_rad,omega,OMega,days2syr(P))
elts0 = get_elements(main_file)
a0 = elts0[0]
e0 = elts0[1]
i0 = elts0[2]
OMega0 = elts0[3]
omega0 = elts0[4]
P0 = elts0[5]

print('eccentricity e = ',round(e,5),e0)
print('semi-major axis, a = ',round(a,5),a0)
print('Period is',round(P,4),'days','or',round(days2syr(P),4),'sid.years', P0)
print('Time of perihelion passage: ',Time.strftime(T_p,"%d-%b-%Y %H:%M:%s"))
print('i = ',round(np.rad2deg(i_rad),5),i0)
print('omega = ',round(np.rad2deg(omega),5),omega0)
print('OMega = ',round(np.rad2deg(OMega),5),OMega0)

# EPHEMERIS
elts = np.array([a,e,i_rad,OMega,omega,days2syr(P)])

t0 = Time.strptime('2020-Aug-10','%Y-%b-%d',scale='utc')
t0 = t0.tt
M0 = ((2*np.pi/(P))*(t0-T_p).value)
M0 = np.rad2deg(M0)
M0 = Angle(M0*u.deg)
print('M0 is value: ',round((M0.value),4))
N = 10 # no. of coordinates to be generated

a = a*u.au
inc = Angle(np.rad2deg(i_rad)*u.deg)
OM = Angle(np.rad2deg(OMega)*u.deg)
w = Angle(np.rad2deg(omega)*u.deg)

step = 1
ra_h,dec_d,D,time = ephemeris(a,e,inc,OM,w,M0,t0,N,step)
cd0,cd1,cd2 = ra_h,dec_d,D
T_str = time2str(time)

eph_file = 'ephemeris-'+num+'.csv'
np.savetxt(eph_file, np.transpose([T_str,cd0,cd1,cd2]),delimiter=';',fmt="%s")
print('Ephemeris table written to',eph_file)


eccentricity e =  0.47981 0.4798327745913477
semi-major axis, a =  2.63821 2.634149536807631
Period is 1565.1701 days or 4.2851 sid.years 4.27532
Time of perihelion passage:  13-Sep-2019 21:52:1568391778
i =  13.19317 13.19480172515017
omega =  152.22245 152.088773087469
OMega =  213.2617 213.2654921364977
M0 is value:  76.1528
Ephemeris table written to ephemeris-QB3.csv


# Orbital Elements table

In [50]:
def elts_table(elts0,elts,filename):
    a0 = round(elts0[0],4)
    e0 = round(elts0[1],4)
    i0 = round(elts0[2],4)
    OMega0 = round(elts0[3],4)
    omega0 = round(elts0[4],4)
    P0 = round(elts0[5],4)
    
    a = elts[0]
    e = elts[1]
    i = elts[2]
    OMega = elts[3]
    omega = elts[4]
    P = elts[5]
    
    acc_a, acc_e, acc_i, acc_w,acc_OM,acc_P = error_check(a,e,i_rad,omega,OMega,P)
    
    x = PrettyTable()
    x.field_names = ["Element","Pipeline Output", "JPL value", "Accuracy"]
    x.add_row(['A',round(a,4),a0,acc_a])
    x.add_row(['EC',round(e,4),e0,acc_e])
    x.add_row(['IN',round(np.rad2deg(i),4),i0,acc_i])
    x.add_row(['OM',round(np.rad2deg(OMega),4),OMega0,acc_OM])
    x.add_row(['W',round(np.rad2deg(omega),4),omega0,acc_w])
    x.add_row(['PER',round(P,4),P0,acc_P])
    print('OBJECT: ',filename[:-4],'\n',x)
    return None

elts_table(elts0,elts,main_file)

OBJECT:  2019-QB3 
 +---------+-----------------+-----------+----------+
| Element | Pipeline Output | JPL value | Accuracy |
+---------+-----------------+-----------+----------+
|    A    |      2.6382     |   2.6341  |  99.846  |
|    EC   |      0.4798     |   0.4798  |  99.996  |
|    IN   |     13.1932     |  13.1948  |  99.988  |
|    OM   |     213.2617    |  213.2655 |  99.998  |
|    W    |     152.2224    |  152.0888 |  99.912  |
|   PER   |      4.2851     |   4.2753  |  99.771  |
+---------+-----------------+-----------+----------+
